In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from torch.cuda.amp import autocast, GradScaler
import pandas as pd
import csv
import seaborn as sns

In [2]:
train = 'Pictures/lab_3/dog-breed-identification/train/'
test = 'Pictures/lab_3/dog-breed-identification/test/'
csv_file_train = 'Pictures/lab_3/dog-breed-identification/labels.csv'

In [ ]:
read_csv_file_train = pd.read_csv(csv_file_train)
counter = {}
for breed in read_csv_file_train['breed'][1:]:
     counter[breed] = counter.get(breed, 0) + 1
print(counter)
X = [key for key in counter.keys()]
Y = [value for value in counter.values()]
sns.barplot(x=X, y=Y)

In [3]:
breed_to_idx = {breed: i for i, breed in enumerate(pd.read_csv(csv_file_train)['breed'].unique())}

In [4]:
class DatasetToClass(torch.utils.data.Dataset):
    def __init__(self, pictures, csv_file):
        super().__init__()
        self.pictures = pictures
        self.data = pd.read_csv(csv_file)
        self.pictures_list = os.listdir(pictures)
        
    def __len__(self):
        return len(self.pictures_list)
    
    def __getitem__(self, index):
        img_name = self.data.iloc[index, 0]
        img_path = os.path.join(self.pictures, str(img_name)+'.jpg')
        label = self.data.iloc[index, 1]
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        
        img = torch.from_numpy(img)
        label = torch.tensor(breed_to_idx[label])
        
        return img, label

In [5]:
train_breed_dogs = DatasetToClass(train, csv_file_train)

In [6]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_breed_dogs, batch_size=batch_size)

In [7]:
class DogBreedClassification(nn.Module):
    def __init__(self, num_classes):
        super(DogBreedClassification, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 120)
        self.fc2 = nn.Linear(120, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
num_breeds = 120
model = DogBreedClassification(num_breeds)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 13
device = torch.device('cuda')
model = model.to(device)

In [10]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        outputs = model(images.to(device))
        loss = criterion(outputs, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
            
    epoch_loss = running_loss / len(train_loader)    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

  0%|          | 0/320 [00:00<?, ?it/s]

Epoch [1/13], Loss: 4.7928


  0%|          | 0/320 [00:00<?, ?it/s]

Epoch [2/13], Loss: 4.7779


  0%|          | 0/320 [00:00<?, ?it/s]

...

Epoch [11/13], Loss: 4.2459


  0%|          | 0/320 [00:00<?, ?it/s]

Epoch [12/13], Loss: 4.2028


  0%|          | 0/320 [00:00<?, ?it/s]

Epoch [13/13], Loss: 4.1617


In [11]:
model.eval()
test_images = os.listdir(test)
results = []

with torch.no_grad():
    for image_name in tqdm(test_images):
        img_path = os.path.join(test, image_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        
        img = torch.from_numpy(img)
        
        outputs = model(img.to(device))
        probabilities = torch.softmax(outputs, dim=1).squeeze().tolist()
        
        result = [image_name] + probabilities
        results.append(result)
        
breed_names = pd.read_csv(csv_file_train).iloc[:, 1].unique()
header = ['image_name'] + list(breed_names)

with open('Pictures/lab_3/results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(results)    

  0%|          | 0/10357 [00:00<?, ?it/s]

In [12]:
results_df = pd.read_csv('Pictures/lab_3/results.csv')
for i in range(10357):
    dict_breed = zip([k for k in results_df[:]][1:], [k for k in results_df.iloc[i, :]][1:])
    mx_ = max(dict_breed, key=lambda x: x[1])
    breed, probability = mx_[0], mx_[1]
    print(results_df.iloc[i, 0], breed, probability, sep='    ')

000621fb3cbb32d8935728e48679680e.jpg    blenheim_spaniel    0.0531169511377811
00102ee9d8eb90812350685311fe5890.jpg    sealyham_terrier    0.1028239876031875
0012a730dfa437f5f3613fb75efcd4ce.jpg    collie    0.0310921818017959
001510bc8570bbeee98c8d80c8a95ec1.jpg    afghan_hound    0.0280606746673584
001a5f3114548acdefa3d4da05474c2e.jpg    great_pyrenees    0.0384972132742404
00225dcd3e4d2410dd53239f95c0352f.jpg    leonberg    0.0366189293563365
002c2a3117c2193b4d26400ce431eebd.jpg    irish_setter    0.0817462876439094
002c58d413a521ae8d1a5daeb35fc803.jpg    bedlington_terrier    0.094241976737976
002f80396f1e3db687c5932d7978b196.jpg    norfolk_terrier    0.0270092617720365
0036c6bcec6031be9e62a257b1c3c442.jpg    otterhound    0.0523656122386455
0041940322116ae58c38130f5a6f71f9.jpg    leonberg    0.0274431332945823
0042d6bf3e5f3700865886db32689436.jpg    samoyed    0.0797009766101837
004476c96f575879af4af471af65cae8.jpg    curly-coated_retriever    0.0347906835377216
00485d47de966a9437

In [14]:
device = torch.device('cuda:0')
model = tv.models.resnet50(weights=tv.models.ResNet50_Weights.IMAGENET1K_V1)
#model = tv.models.resnet50(pretrained=True)
model = model.to(device)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 120)
model = model.to(device)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        
        outputs = model(images.to(device))
        optimizer.zero_grad()
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

model.eval()
test_images = os.listdir(test)
breed_res = []

with torch.no_grad():
    for image_name in tqdm(test_images):
        img_path = os.path.join(test, image_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        
        img = torch.from_numpy(img)
        img = img.to(device)
        outputs = model(img.unsqueeze(0))
        probabilities = torch.softmax(outputs, dim=1).squeeze().tolist()
        
        result = [image_name] + probabilities
        breed_res.append(result)

  0%|          | 0/320 [00:00<?, ?it/s]

Epoch [1/10], Loss: 4.8354


  0%|          | 0/320 [00:00<?, ?it/s]

Epoch [2/10], Loss: 4.8354


  0%|          | 0/320 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
breed_names_resnet = pd.read_csv(csv_file_train).iloc[:, 1].unique()
header = ['image_name'] + list(breed_names)

with open('Pictures/lab_3/results_resnet.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(results)  

In [16]:
results_df = pd.read_csv('Pictures/lab_3/results_resnet.csv')
for i in range(10357):
    dict_breed = zip([k for k in results_df[:]][1:], [k for k in results_df.iloc[i, :]][1:])
    mx_ = max(dict_breed, key=lambda x: x[1])
    breed, probability = mx_[0], mx_[1]
    print(results_df.iloc[i, 0], breed, probability, sep='    ')

000621fb3cbb32d8935728e48679680e.jpg    blenheim_spaniel    0.0531169511377811
00102ee9d8eb90812350685311fe5890.jpg    sealyham_terrier    0.1028239876031875
0012a730dfa437f5f3613fb75efcd4ce.jpg    collie    0.0310921818017959
001510bc8570bbeee98c8d80c8a95ec1.jpg    afghan_hound    0.0280606746673584
001a5f3114548acdefa3d4da05474c2e.jpg    great_pyrenees    0.0384972132742404
00225dcd3e4d2410dd53239f95c0352f.jpg    leonberg    0.0366189293563365
002c2a3117c2193b4d26400ce431eebd.jpg    irish_setter    0.0817462876439094
002c58d413a521ae8d1a5daeb35fc803.jpg    bedlington_terrier    0.094241976737976
002f80396f1e3db687c5932d7978b196.jpg    norfolk_terrier    0.0270092617720365
0036c6bcec6031be9e62a257b1c3c442.jpg    otterhound    0.0523656122386455
0041940322116ae58c38130f5a6f71f9.jpg    leonberg    0.0274431332945823
0042d6bf3e5f3700865886db32689436.jpg    samoyed    0.0797009766101837
004476c96f575879af4af471af65cae8.jpg    curly-coated_retriever    0.0347906835377216
00485d47de966a9437

In [ ]:
torch.cuda.is_available()